In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from IPython.display import display

In [23]:
# ---------------------------------
# Code  Description
# ---------------------------------
# e	The value has been edited or estimated by USGS personnel.
# <	The Value is known to be less than reported value.
# >	The value is known to be greater than reported value.
# R	Records for these data have been revised.
# A	Approved for publication -- Processing and review completed.
# P	Provisional data subject to revision.

In [24]:
#Testing with percentiles
def twenty_per_less(num):
    
    print("20 percent less than ", num, "is" ,(num * 0.8))

def CCDF_baseflow(file_name):

    df = pd.read_csv(file_name+".csv")

    print("Baseflow Information: \n")
    
    august_data = df[df['month_nu'] == 8]

    average_august_value = august_data['mean_va'].mean()
    print("Avg Aug Val: ", average_august_value)

    per_fifty_average_august_value = august_data['p50_va'].mean()
    print("50% Avg Aug Val: ", per_fifty_average_august_value)

    twenty_per_less(average_august_value)
    twenty_per_less(per_fifty_average_august_value)


In [25]:
#Spring Storms Data (percentiles)
def CCDF_spring_storms(file_name):

    df = pd.read_csv(file_name+".csv")

    print("Spring Storms Information: \n")

    eighty_per = df.loc[(df['month_nu'] >= 4) & (df['month_nu'] <= 6) & (df['day_nu'] >= 15) & (df['day_nu'] <= 30), 'p80_va']
    twenty_per = df.loc[(df['month_nu'] >= 4) & (df['month_nu'] <= 6) & (df['day_nu'] >= 15) & (df['day_nu'] <= 30), 'p20_va']

    eighty_mean_vals = eighty_per.mean()
    twenty_mean_vals = twenty_per.mean()

    print(eighty_mean_vals)
    print(twenty_mean_vals)



In [26]:
#Testing with Average Daily Exceedance Flow Equation
def CCDF_aug_avg_daily_exceedance_flow(file_name):

    df = pd.read_csv(file_name + ".csv")
    
    #august_data = df[df['month_nu'] == 8]
    august_data = df[df['month_nu'] == 8].copy()

    #august_data['ranked_column_m'] = august_data['mean_va'].rank(ascending=False)

    august_data['ranked_column_m'] = august_data.sort_values('mean_va', ascending=False)

    df['numbered_col'] = range(1, len(df) + 1)

    #sort the values from highest to lowest
    #make a new column numbered from 1 to the end of the number or rows (m)
    #new column with the calculated P


    print("ranked column m:", august_data['ranked_column_m'].head())
    print("mean values: ", august_data['mean_va'].head())
    print("????", august_data['mean_va'].rank(ascending=False))

    
    #august_data.loc[:, 'ranked_column_m'] = august_data['mean_va'].rank(ascending=False)
    #df['ranked_column_m'] = df['mean_va'].rank(ascending=False)
    
    n = len(august_data)

    df['P_Value'] = range(1, len(df) + 1)

    P = 0

    for i in n:
        m = august_data.iloc[i]['ranked_column_m']
        #august_data['ranked_column_m'] = august_data['mean_va']
        P = 100 * (m / (n + 1))
        df.at[i, 'P_Value'] = P
        #print(df['P_Value'])

In [27]:
#Testing with Average Daily Exceedance Flow Equation
def CCDF_spring_flow_avg_daily_exceedance_flow(file_name):

    df = pd.read_csv(file_name + ".csv")

    spring_data = df.loc[(df['month_nu'] >= 4) & (df['month_nu'] <= 6) & (df['day_nu'] >= 15) & (df['day_nu'] <= 30), 'mean_va']

    spring_data['ranked_column_m'] = spring_data['mean_va'].rank(ascending=False)
    
    #august_data.loc[:, 'ranked_column_m'] = august_data['mean_va'].rank(ascending=False)
    #df['ranked_column_m'] = df['mean_va'].rank(ascending=False)
    
    n = len(spring_data)

    P = 0

    for i in range(n):
        m = spring_data.iloc[i]['ranked_column_m']
        P += 100 * (m / (n + 1))
    
    print (P)

In [28]:
CCDF_baseflow('wallstreet_data')
CCDF_spring_storms('wallstreet_data')

CCDF_aug_avg_daily_exceedance_flow('wallstreet_data')
#CCDF_spring_flow_avg_daily_exceedance_flow('wallstreet_data')

Baseflow Information: 

Avg Aug Val:  310.258064516129
50% Avg Aug Val:  267.61290322580646
20 percent less than  310.258064516129 is 248.20645161290324
20 percent less than  267.61290322580646 is 214.0903225806452
Spring Storms Information: 

1468.0
379.3333333333333


ValueError: Cannot set a DataFrame with multiple columns to the single column ranked_column_m

In [ ]:
#Plotting CCDF
def CCDF_plot(file_name, x, y):

    df = pd.read_csv(file_name + ".csv")

    fig, ax = plt.subplots(figsize=(16, 6))
    ax.plot(df[x], df[y], marker='o', linestyle='-')
    #plot scatter
    plt.title('CCDF of ' + file_name)
    plt.xlabel('Flow Duration Interval (or) Exceedance Probability (%)') #P value
    plt.ylabel('Mean Daily Flow (cfs)')
    plt.grid(True)

    return plt.show()